In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('./Churn_Modelling.csv')
print(data.head())

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

In [2]:
X = data.iloc[:, 3:13]
y = data.iloc[:, 13]

In [3]:
geo = pd.get_dummies(X['Geography'],drop_first=True)
gen = pd.get_dummies(X['Gender'], drop_first=True)

In [4]:
Geography = geo.copy()
Gender = gen.copy()

X = X.drop(['Gender','Geography'], axis = 1)
X = pd.concat([X,Gender, Geography],axis=1)

print(X.head(5))


   CreditScore  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0          619   42       2       0.00              1          1   
1          608   41       1   83807.86              1          0   
2          502   42       8  159660.80              3          1   
3          699   39       1       0.00              2          0   
4          850   43       2  125510.82              1          1   

   IsActiveMember  EstimatedSalary  Male  Germany  Spain  
0               1        101348.88     0        0      0  
1               1        112542.58     0        0      1  
2               0        113931.57     0        0      0  
3               0         93826.63     0        0      0  
4               1         79084.10     0        0      1  


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout


Using TensorFlow backend.


In [7]:
classifier = Sequential()
#Added input layer
classifier.add(Dense(units=10, kernel_initializer = 'he_uniform', activation='relu', input_dim=X_train.shape[1]))
#Adding Hidden layer
classifier.add(Dense(units=6, kernel_initializer = 'he_uniform', activation = 'relu'))
#Adding Output Layer
classifier.add(Dense(units=1, kernel_initializer='glorot_uniform', activation = 'sigmoid'))

classifier.compile(optimizer='Adamax', loss='binary_crossentropy', metrics=['accuracy'])


In [8]:
model_result = classifier.fit(X_train, y_train, validation_split=0.33, batch_size=10, epochs=100)

Train on 5359 samples, validate on 2641 samples
Epoch 1/100
5359/5359 [==============================] - 2s 374us/step - loss: 0.9140 - accuracy: 0.5225 - val_loss: 0.5446 - val_accuracy: 0.7914
Epoch 2/100
5359/5359 [==============================] - 2s 338us/step - loss: 0.5052 - accuracy: 0.7966 - val_loss: 0.4987 - val_accuracy: 0.7948
Epoch 3/100
5359/5359 [==============================] - 2s 396us/step - loss: 0.4753 - accuracy: 0.7974 - val_loss: 0.4758 - val_accuracy: 0.7955
Epoch 4/100
5359/5359 [==============================] - 2s 391us/step - loss: 0.4563 - accuracy: 0.7983 - val_loss: 0.4615 - val_accuracy: 0.7978
Epoch 5/100
5359/5359 [==============================] - 2s 400us/step - loss: 0.4426 - accuracy: 0.8057 - val_loss: 0.4517 - val_accuracy: 0.8001
Epoch 6/100
5359/5359 [==============================] - 2s 414us/step - loss: 0.4325 - accuracy: 0.8100 - val_loss: 0.4454 - val_accuracy: 0.8008
Epoch 7/100
5359/5359 [==============================] - 2s 421us/step

In [9]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred>0.5)

from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)
print("Score is : "+str(score))

Score is : 0.864


In [10]:
# Performing Hyper Parameter Tuning

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from keras.wrappers.scikit_learn import KerasClassifier

def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
    model.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
    model.compile(optimizer='Adamax', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [11]:
from sklearn.model_selection import GridSearchCV
model = KerasClassifier(build_fn=create_model)

batches = [128]
epochs = [30]
layers = [[20], [40,20], [45,30,15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation = activations, batch_size=batches, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [12]:
#Commeneted out as it was performed
grid_result = grid.fit(X_train, y_train)

C:\Users\sahil\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/30
5333/5333 [==============================] - 0s 78us/step - loss: 0.6198 - accuracy: 0.6855
Epoch 2/30
5333/5333 [==============================] - 0s 24us/step - loss: 0.5340 - accuracy: 0.7922
Epoch 3/30
5333/5333 [==============================] - 0s 23us/step - loss: 0.5102 - accuracy: 0.7973
Epoch 4/30
5333/5333 [==============================] - 0s 23us/step - loss: 0.4961 - accuracy: 0.7969
Epoch 5/30
5333/5333 [==============================] - 0s 23us/step - loss: 0.4849 - accuracy: 0.7986
Epoch 6/30
5333/5333 [==============================] - 0s 24us/step - loss: 0.4797 - accuracy: 0.7979
Epoch 7/30
5333/5333 [==============================] - 0s 24us/step - loss: 0.4776 - accuracy: 0.7977
Epoch 8/30
5333/5333 [==============================] - 0s 24us/step - loss: 0.4711 - accuracy: 0.7994
Epoch 9/30
5333/5333 [==============================] - 0s 23us/step - loss: 0.4636 - accuracy: 0.7994
Epoch 10/30
5333/5333 [==============================] - 0s 23us/step - l

In [13]:
best_score = grid_result.best_score_
best_param = grid_result.best_params_
best_estimator = grid_result.best_estimator_

print("Best Score After Hyper Paramter Tuning : "+str(best_score))
print("Best Parametres are : "+str(best_param))

Best Score After Hyper Paramter Tuning : 0.8482500056400896
Best Parametres are : {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [45, 30, 15]}


In [14]:
tuned_classifier = Sequential()

tuned_classifier.add(Dense(units=40, kernel_initializer='he_uniform', activation='relu', input_dim=X_train.shape[1]))
tuned_classifier.add(Dropout(0.3))

tuned_classifier.add(Dense(units=20, kernel_initializer = 'he_uniform', activation='relu'))
tuned_classifier.add(Dropout(0.3))

tuned_classifier.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))

tuned_classifier.compile(optimizer='Adamax', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
tuned_model_history = tuned_classifier.fit(X_train, y_train, validation_split=0.33, batch_size=128, epochs=40)

Train on 5359 samples, validate on 2641 samples
Epoch 1/40
5359/5359 [==============================] - 1s 116us/step - loss: 0.6968 - accuracy: 0.6363 - val_loss: 0.5226 - val_accuracy: 0.7925
Epoch 2/40
5359/5359 [==============================] - 0s 36us/step - loss: 0.5786 - accuracy: 0.7492 - val_loss: 0.4934 - val_accuracy: 0.7955
Epoch 3/40
5359/5359 [==============================] - 0s 36us/step - loss: 0.5424 - accuracy: 0.7664 - val_loss: 0.4751 - val_accuracy: 0.7963
Epoch 4/40
5359/5359 [==============================] - 0s 35us/step - loss: 0.5287 - accuracy: 0.7725 - val_loss: 0.4642 - val_accuracy: 0.7974
Epoch 5/40
5359/5359 [==============================] - 0s 37us/step - loss: 0.5029 - accuracy: 0.7873 - val_loss: 0.4564 - val_accuracy: 0.7993
Epoch 6/40
5359/5359 [==============================] - 0s 36us/step - loss: 0.4943 - accuracy: 0.7867 - val_loss: 0.4508 - val_accuracy: 0.8016
Epoch 7/40
5359/5359 [==============================] - 0s 36us/step - loss: 0.49

In [16]:
tuned_score = accuracy_score(y_pred, y_test)
print("Score after Tuning the model is : "+str(tuned_score))

filename = 'churn-model-85'
import pickle
pickle.dump(model, open(filename, 'wb'))
print("Model saved succesfully!")

Score after Tuning the model is : 0.864
Model saved succesfully!
